### Note: Duplicate this notebook so you can run mutliple processes with different api keys concurrently

The following code cell ensures that the underlying `.py` file gets automatically reloaded after you save any changes.  

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd    
from model import initialize_model, call_llm
import time

together_token2 = "dddb92d11f5d51d18e42a6b0ac35a68c2d6cd098860c571208d8aa44def733c8" # The api key from your second together account
model_name = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

/Users/jasonchung/Downloads/nlp/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    return data

In [4]:
data = preprocess("./data/train.jsonl")

In [5]:
first_partition = data[:1365]
second_partition = data[1365:2730]
third_partition = data[2730:4095]
fourth_partition = data[4095:5460]
fifth_partition = data[5460:6825]
sixth_partition = data[6825:]

In [8]:
labels = []
reasonings = []
raw_output = []
ids = []
client, _ = initialize_model("", defined_api_key=together_token2)
last_time = time.time()

# TODO Change len(data) to len(your_partition)
for i in range(574, len(sixth_partition)):
    if i > 0 and i % 60 == 0:
        print(i)
        current_time = time.time()
        time.sleep(max(0, 63 - (current_time - last_time)))
        last_time = time.time()

    # TODO Change data to your partition
    current_data = sixth_partition.iloc[i]
    ids.append(current_data.unique_id)
    classification, reasoning = call_llm(client, model_name, current_data.sectionName, current_data.string)
    raw_output.append(classification)
    labels.append(classification)
    reasonings.append(reasoning)
    

id='noVuYHR-3NKUce-92a1a3f1f99b6c00' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743610688 model='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=13957360492917800000, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='{"classification": "background", "reasoning": "The text provides a list of references to previous studies on various types of wastewater, which suggests that it is setting the stage for the current research by providing context and reviewing existing literature, a typical characteristic of a background or introduction section in a scientific paper."}', tool_calls=[]))] prompt=[] usage=UsageData(prompt_tokens=263, completion_tokens=63, total_tokens=326, cached_tokens=0)
id='noVuZSY-4Yz4kd-92a1a40c38d56c00' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743610692 model='meta-llama/Llama-3.3-70B-In

In [6]:
df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
df.to_csv("fifth_partition_gemma.csv")